In [1]:
import numpy as np
import pandas as pd
from datetime import datetime,date,time
from dateutil.relativedelta import relativedelta


d:\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
d:\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
d:\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


### **A) Coleta dos Dados de Direção do Vento**

In [2]:
source_file = '../../../../PrevisaoVento/Data/raw/Variaveis_EPE/BAUET2_CAETITÉ_2_PRE_Verif_EPE_edit.txt'

In [3]:
df_coleta = pd.read_csv(source_file, delimiter = ';' , header=None )

### **B) Sobre os Dados no Patio 1**

#### <span style="color:#DC143C">**B1.Formato dos Dados**

In [4]:
df_coleta.shape # formato da matriz

(1622, 49)

#### <span style="color:#DC143C">**B2.Visao do Dataframe**

In [5]:
df_coleta.head(5)  # visão de 5 linhas

,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
0,20170101,882.334,882.334,882.333,882.333,882.0,882.0,882.000,882.000,882.333,...,883.0,883.000,883.0,883.0,884.0,884.000,884.0,884.000,883.333,883.0
1,20170102,883.000,882.667,882.000,882.000,882.0,882.0,881.667,881.333,882.000,...,881.0,881.000,881.0,881.0,881.0,881.333,881.0,881.667,882.000,882.0
2,20170103,882.000,881.000,881.000,881.000,881.0,881.0,881.000,881.000,881.000,...,881.0,881.000,881.0,881.0,882.0,882.000,882.0,882.000,882.000,882.0
3,20170104,882.000,882.000,882.000,882.000,882.0,882.0,882.000,882.000,882.000,...,882.0,882.667,883.0,883.0,883.0,883.000,884.0,884.000,884.000,884.0
4,20170105,883.000,883.000,883.000,883.000,883.0,883.0,883.000,883.000,883.000,...,883.0,883.000,883.0,883.0,883.0,884.000,884.0,884.000,884.000,884.0


### **C) Extração os Dados de 2017**

#### <span style="color:#DC143C"> **C1. Converter a primeira coluna em Data**

In [6]:
df_coleta[0]= pd.to_datetime(df_coleta[0], format='%Y%m%d')

#### <span style="color:#DC143C"> **C2.Remove registros fora do Intervalo**

In [7]:
df_coleta = df_coleta.drop(df_coleta[df_coleta[0] > '2018-01-02'].index)
df_coleta = df_coleta.drop(df_coleta[df_coleta[0] < '2017-01-01'].index)

In [8]:
# Inclui colunas de Latitude e Longitude
df_coleta['longitude'] = -42.75
df_coleta['latitude']  = -14.25 

In [9]:
# Transfere as 2 ultimas colunas para a primeira posição da lista
cols = list(df_coleta)
cols = [cols[-1]] + cols[:-1]
df_coleta = df_coleta[cols]

cols = list(df_coleta)
cols = [cols[-1]] + cols[:-1]
df_coleta = df_coleta[cols]

In [10]:
df_coleta = df_coleta.reset_index()

### **D) Formatação dos Dados**

#### <span style="color:#DC143C"> **D1. Migrar os dados para o Formato**

In [11]:
frente = ['Longitude', 'Latitude', 'Data', 'Valor30_1', 'Valor30_2']  # Cabeçalho
df_pressao = pd.DataFrame(columns=frente)  # Criação de um dataframe vazio
    
Linha1    =  df_coleta.shape[0] 
Colunas1  =  df_coleta.shape[1] 

for lin2 in range(Linha1):  
    refhora = 0
    
    for col in range (Colunas1):        
        if col > 0:            
        
            if col % 2 != 0: # Se o número (col) for impar                   
               
                if col < 49:
                    
                    Data = df_coleta.loc[lin2,0]
                                  
                    hora = time(hour=refhora, minute=0, second=0)            
                    dataCompleta = datetime.combine(Data, hora)
                    #print(dataCompleta)
                    lin_alvo = [df_coleta.loc[lin2,'longitude'], df_coleta.loc[lin2,'latitude'], dataCompleta , df_coleta.loc[lin2,col], df_coleta.loc[lin2,col+1]] 
                    # print(lin_alvo)
                    df_pressao.loc[len(df_pressao)] = lin_alvo  # adding a row
                         
                refhora = refhora + 1          
                


#### <span style="color:#DC143C"> **D2. Informações sobre os Tipos de Dados**

In [12]:
df_pressao.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8808 entries, 0 to 8807
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Longitude  8808 non-null   float64       
 1   Latitude   8808 non-null   float64       
 2   Data       8808 non-null   datetime64[ns]
 3   Valor30_1  8808 non-null   float64       
 4   Valor30_2  8808 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 412.9 KB


In [13]:
df_pressao.shape

(8808, 5)

In [14]:
df_pressao = df_pressao.reset_index()

#### <span style="color:#DC143C"> **D3. Agregar Magnitude para 1H**

In [15]:
df_pressao['pressao_Verif'] = (df_pressao['Valor30_1'] + df_pressao['Valor30_2']) / 2

In [16]:
# Formatar o resultado com 2 casas decimais
df_pressao['pressao_Verif'] = df_pressao['pressao_Verif'].round(decimals=3)

In [17]:
df_pressao = df_pressao.drop('Valor30_1', 1)
df_pressao = df_pressao.drop('Valor30_2', 1)

In [18]:
df_pressao.head(-10)

,index,Longitude,Latitude,Data,pressao_Verif
0,0,-42.75,-14.25,2017-01-01 00:00:00,882.334
1,1,-42.75,-14.25,2017-01-01 01:00:00,882.333
2,2,-42.75,-14.25,2017-01-01 02:00:00,882.000
3,3,-42.75,-14.25,2017-01-01 03:00:00,882.000
4,4,-42.75,-14.25,2017-01-01 04:00:00,882.666
...,...,...,...,...,...
8793,8793,-42.75,-14.25,2018-01-02 09:00:00,882.000
8794,8794,-42.75,-14.25,2018-01-02 10:00:00,882.000
8795,8795,-42.75,-14.25,2018-01-02 11:00:00,882.000
8796,8796,-42.75,-14.25,2018-01-02 12:00:00,881.166


### **E. Exporta Resultados**

In [19]:
output_file = '../../../../PrevisaoVento/Data/process/cenario2/pressao_ajustada.csv'
df_pressao.to_csv(output_file, sep=';', encoding='utf-8')